In [97]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import os
from datetime import datetime
import pytz
import locale
from time import time

In [98]:
# Definição das variáveis
loja = 1

In [99]:
def format_number(value):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')  # Set your desired locale
    formatted_value = locale.format('%.2f', value, grouping=True)
    formatted_value = str(formatted_value).replace('.', '')
    return formatted_value

In [100]:
def municipio_para_uf(codigo_municipal):
    # Dicionário de mapeamento de códigos municipais para siglas de UF
    codigo_uf = {
        "11": "RO", "12": "AC", "13": "AM", "14": "RR", "15": "PA", "16": "AP", "17": "TO",
        "21": "MA", "22": "PI", "23": "CE", "24": "RN", "25": "PB", "26": "PE", "27": "AL",
        "28": "SE", "29": "BA", "31": "MG", "32": "ES", "33": "RJ", "35": "SP", "41": "PR",
        "42": "SC", "43": "RS", "50": "MS", "51": "MT", "52": "GO", "53": "DF"
    }
    
    # Extrai os dois primeiros dígitos do código municipal para obter o código da UF
    codigo_uf_municipal = codigo_municipal[:2]
    
    # Verifica se o código da UF está no dicionário
    if codigo_uf_municipal in codigo_uf:
        return codigo_uf[codigo_uf_municipal]
    else:
        return "Código municipal não encontrado"

In [101]:
def ler_ultima_serie_utilizada():
    try:
        with open(f'C:/Bots/Bot_transmissao/Geracao CAT/ultima_serie_utilizada.txt', 'r') as arquivo:
            ultima_serie = int(arquivo.read())
    except FileNotFoundError:
        # Se o arquivo não existir, retorna 0 como último número utilizado
        ultima_serie = 890
    return ultima_serie

In [102]:
def ler_ultimo_numero_utilizado():
    try:
        with open(f'C:/Bots/Bot_transmissao/Geracao CAT/Loja_{loja}/ultimo_numero_utilizado.txt', 'r') as arquivo:
            ultimo_numero = int(arquivo.read())
    except FileNotFoundError:
        # Se o arquivo não existir, retorna 0 como último número utilizado
        ultimo_numero = 0
    return ultimo_numero

In [103]:
# Função para escrever o último número utilizado em um arquivo
def escrever_ultimo_numero_utilizado(ultimo_numero):
    with open(f'C:/Bots/Bot_transmissao/Geracao CAT/Loja_{loja}/ultimo_numero_utilizado.txt', 'w') as arquivo:
        arquivo.write(str(ultimo_numero))

In [104]:
# Função para escrever o último número utilizado em um arquivo
def escrever_ultima_serie_utilizada(ultima_serie):
    with open('C:/Bots/Bot_transmissao/Geracao CAT/ultima_serie_utilizada.txt', 'w') as arquivo:
        arquivo.write(str(ultima_serie))

In [105]:
diretorio_final = f'C:/Bots/Bot_transmissao/Geracao CAT/Loja_{loja}/TXTs_complementar'

periodos = [2]

ultima_serie_utilizada = ler_ultima_serie_utilizada()

for periodo in periodos:
    
    # Leitura da complementar da loja

    comp = pd.read_excel(f'C:/Bots/Bot_transmissao/Geracao CAT/Loja_{loja}/complementar_final_loja_{loja}_p{periodo}_v1.xlsx')
    comp['Item NFE Origem'] = comp['Item NFE Origem'].astype(str)
    
    comp['Valor Complementar'] = comp['ST Retido - Base de Cálculo'].str.replace(',', '.').astype(float)
    comp = comp[comp['Valor Complementar'] > 0]
    
    # Leitura da Tabela 2.1

    tab_2_1 = pd.read_csv(f'C:/Bots/Bot_transmissao/Geracao CAT/Loja_{loja}/loja_{loja}_tabela_2_1_p{periodo}_v1.csv', dtype=str)
    tab_2_1['NUM_ITEM'] = tab_2_1['NUM_ITEM'].astype(str)
    
    merged = comp.merge(tab_2_1,
                   left_on=['Chave Acesso NFe', 'Item NFE Origem'],
                   right_on=['CHV_DOC', 'NUM_ITEM'])


    merged['CEST'] = merged['CEST'].astype(str).str.replace(r'\.0$', '', regex=True)
    merged['CEST'] = merged['CEST'].replace('nan', '')
    merged['N C M'] = merged['N C M'].astype(str).str.replace(r'\.0$', '', regex=True)
    merged['EAN'] = merged['EAN'].astype(str).str.replace(r'\.0$', '', regex=True)
    merged['EAN'] = merged['EAN'].replace('nan', '')
    merged['ALIQUOTA_x'] = merged['ALIQUOTA_x'].astype(int)
    merged['ICMS ST Retido- Valor'] = merged['ICMS ST Retido- Valor'].astype(float).map(format_number)
    merged['ST Retido - Base de Cálculo'] = merged['ST Retido - Base de Cálculo'].str.replace(',', '.').astype(float).map(format_number)
    merged['CNPJ DESTINATARIO'] = merged['CNPJ DESTINATARIO'].astype(str).str.replace(r'\.0$', '', regex=True)
    merged['ICMS ST Retido- Valor'] = merged['ICMS ST Retido- Valor'].str.replace(',', '.')
    merged['ST Retido - Base de Cálculo'] = merged['ST Retido - Base de Cálculo'].str.replace(',', '.')
    merged['UNIDADE'] = merged['UNIDADE'].str.strip()
    
    lojas_enderecos = pd.read_excel(f'C:/Bots/Bot_transmissao/Geracao CAT/Referencias/Endereços_P{periodo}.xlsx', dtype=str)
    lojas_enderecos['telefone'] = lojas_enderecos['telefone'].str.replace(' ', '')
    
    
    chaves = list(comp['Chave Acesso NFe'].unique())
    
    for chave in chaves:

        print(chave)
        
        ultimo_numero_utilizado = ler_ultimo_numero_utilizado() 
        
        linhas = []
        # Filtragem das informações necessárias das tabelas
    #     ficha_3_filt = ficha3[ficha3['CHV_DOC'] == chave]
        comp_filt = comp[comp['Chave Acesso NFe'] == chave]
        tab_2_1_filt = tab_2_1[tab_2_1['CHV_DOC'] == chave]
        cnpj = chave[6:20]
        merged_filt = merged[merged['Chave Acesso NFe'] == chave]

        #==============Montagem do TXT==============#
        # 1ª linha

        cNF = np.random.randint(1, 10000000)
        natOp = "COMPLEMENTO DE TRIBUTO"
        mod = 55
        serie = ultima_serie_utilizada - 1
        nNf = ultimo_numero_utilizado + 1
        dhEmi = datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%Y-%m-%dT%H:%M:%S-03:00')
        dhSaiEnt = datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%Y-%m-%dT%H:%M:%S-03:00')
        tpNF = 1
        idDest = 1
        cMunFG = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['codigo municipio'].values[0]
        tplmp = 1
        tpEmis = 1
        cDV = chave[-1]
        tpAmb = 1
        finNFe = 2
        indFinal = 0
        indPres = 0
        indIntermed = ''
        procEmi = 0


        seq = str(datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%y%m%d%H%M%S')) + str(np.random.randint(1,10)) + str(np.random.randint(1,10))
        linha_inicial = f'NOTA FISCAL|1|9999|0001|{seq}|USU-CTR|'
        linhas.append(linha_inicial)

        seq_2 = np.random.randint(10000000, 100000000)

        linha_A = f'A|4.00|NFe|{seq_2}|'
        linha_B = f'B|35|{cNF}|{natOp}|{mod}|{serie}|{nNf}|{dhEmi}|{dhSaiEnt}|{tpNF}|{idDest}|{cMunFG}|{tplmp}|{tpEmis}|{cDV}|{tpAmb}|{finNFe}|{indFinal}|{indPres}|{indIntermed}|{procEmi}|4.0.2.2|||'

        linhas.append(linha_A)
        linhas.append(linha_B)

        # 2º linha

        linha_BA = 'BA|'
        linha_BA02 = f'BA02|{chave}|'

        linhas.append(linha_BA)
        linhas.append(linha_BA02)

        # 3ª linha

        xNomeEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['nome'].values[0]
        xFantEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['nome'].values[0]
        ieEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['ie'].values[0]

        linha_C = f'C|{xNomeEmit}|{xFantEmit}|{ieEmit}||||3|'

        linhas.append(linha_C)

        # 4ª linha
        cnpjEmit = tab_2_1_filt['CHV_DOC'].iloc[0][6:20]

        linha_C02 = f'C02|{cnpjEmit}|'

        linhas.append(linha_C02)

        # 5ª linha

        xLgrEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['endereço'].values[0]
        nroEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['numero'].values[0]
        xCplEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['complemento'].values[0]
        xBairroEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['bairro'].values[0]
        cMunEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['codigo municipio'].values[0]
        xMunEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['cidade'].values[0]
        ufEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['estado'].values[0]
        cepEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['cep'].values[0]
        cPaisEmit = 1058
        xPaisEmit = 'Brasil'
        phoneEmit = lojas_enderecos[lojas_enderecos['cnpj'] == cnpj]['telefone'].values[0]

        linha_C05 = f'C05|{xLgrEmit}|{nroEmit}|{xCplEmit}|{xBairroEmit}|{cMunEmit}|{xMunEmit}|{ufEmit}|{cepEmit}|{cPaisEmit}|{xPaisEmit}|{phoneEmit}|'

        linhas.append(linha_C05)

        # 6ª linha

        xNomeDest = lojas_enderecos[lojas_enderecos['loja'] == f'{loja}']['nome'].values[0]
        indIEDest = 1
        ieDest = lojas_enderecos[lojas_enderecos['loja'] == f'{loja}']['ie'].values[0]

        linha_E = f'E|{xNomeDest}|{indIEDest}|{ieDest}||||'

        linhas.append(linha_E)

        # 7ª linha
        cnpjDest = lojas_enderecos[lojas_enderecos['loja'] == f'{loja}']['cnpj'].values[0]

        linha_E02 = f'E02|{cnpjDest}|'

        linhas.append(linha_E02)

        # 8ª linha

        xLgrDest = lojas_enderecos[lojas_enderecos['cnpj'] == cnpjDest]['endereço'].values[0]
        nroDest = lojas_enderecos[lojas_enderecos['cnpj'] == cnpjDest]['numero'].values[0]
        xCplDest = lojas_enderecos[lojas_enderecos['cnpj'] == cnpjDest]['complemento'].values[0]
        xBairroDest = lojas_enderecos[lojas_enderecos['cnpj'] == cnpjDest]['bairro'].values[0]
        cMunDest = lojas_enderecos[lojas_enderecos['cnpj'] == cnpjDest]['codigo municipio'].values[0]
        xMunDest = lojas_enderecos[lojas_enderecos['cnpj'] == cnpjDest]['cidade'].values[0]
        ufDest = lojas_enderecos[lojas_enderecos['cnpj'] == cnpjDest]['estado'].values[0]
        cepDest = lojas_enderecos[lojas_enderecos['cnpj'] == cnpjDest]['cep'].values[0]
        cPaisDest = 1058
        xPaisDest = 'Brasil'
        phoneDest = lojas_enderecos[lojas_enderecos['cnpj'] == cnpjDest]['telefone'].values[0]

        linha_E05 = f'E05|{xLgrDest}|{nroDest}|{xCplDest}|{xBairroDest}|{cMunDest}|{xMunDest}|{ufDest}|{cepDest}|{cPaisDest}|{xPaisDest}|{phoneDest}|'

        linhas.append(linha_E05)

        num_item = 1

        parte = 1

        # 9ª linha, 11ª linha, 12ª linha
        for i, row in merged_filt.iterrows():

            # Linha 9
            cProd = row['cod novo']
            cEAN = row['EAN']
            xProd = row['DESCRICAO']
            ncm = row['N C M']
            cest = row['CEST']
            cfop = 5403
            uCom = row['UNIDADE']
            qCom = '0.0000'
            vUnCom = '0.00'
            vProd = '0.00'
            cEANTrib = row['EAN']
            uTrib = '0.0000'
            qTrib = '0.0000'
            vUnTrib = '0.00'

            # Linha 11
            if pd.isnull(row['ANVISA']):
                anvisa = ''
            else:
                anvisa = row['ANVISA']

            # Linha 12
            orig = 0
            cst = 60
            vBCSTRet = row['ST Retido - Base de Cálculo'] 
            pst = row['ALIQUOTA_x']
            vICMSSubstituto = 0
            vICMSSTRet = row['ICMS ST Retido- Valor']

            linhas.append(f'H|{num_item}|MENSAGEM:VAL APROXIMADO DOS TRIBUTOS R$ 0.00|')
            linhas.append(f'I|{cProd}|{cEAN}||{xProd}|{ncm}||{cest}|||||{cfop}|{uCom}|{qCom}|{vUnCom}|{vProd}|{cEANTrib}||{uTrib}|{qTrib}|{vUnTrib}|||||1|')

            if not pd.isnull(row['ANVISA']):
                linhas.append(f'K|{anvisa}||0|')

            linhas.append('M|0.00|')
            linhas.append('N|')
            linhas.append(f'N08|{orig}|{cst}|{vBCSTRet}|{pst}|{vICMSSubstituto}|{vICMSSTRet}||||||||')
            linhas.append('Q|')
            linhas.append('Q04|07|')
            linhas.append('S|')
            linhas.append('S04|07|')

            num_item += 1

            if num_item%400 == 1:

                # 13ª linha

                linhas.append('V01||')
                linhas.append('W|')
                linhas.append('W02|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|')

                # 15ª linha
                linhas.append('X|9|')

                # 16ª linha

                tPag = 90
                vPag = 0

                linhas.append('Y|')
                linhas.append('YA|')
                linhas.append(f'YA01||{tPag}||{vPag}|')
                linhas.append('Z|||||||||||||||||||||||')
                linhas.append('ZD|||||||')

                with open(os.path.join(diretorio_final, f'NF_{nNf}_Loja_{loja}.txt'), 'w') as arquivo:
                    arquivo.write('\n'.join(linhas) + '\n')


                linhas = []

                cNF = np.random.randint(1, 100000000)

                seq = str(datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%y%m%d%H%M%S')) + str(np.random.randint(1,10)) + str(np.random.randint(1,10))
                linha_inicial = f'NOTA FISCAL|1|9999|0001|{seq}|USU-CTR|'              

                seq_2 = np.random.randint(10000000, 100000000)

                linha_A = f'A|4.00|NFe|{seq_2}|'
                
                ultimo_numero_utilizado += 1
                nNf = ultimo_numero_utilizado + 1
                linha_B = f'B|35|{cNF}|{natOp}|{mod}|{serie}|{nNf}|{dhEmi}|{dhSaiEnt}|{tpNF}|{idDest}|{cMunFG}|{tplmp}|{tpEmis}|{cDV}|{tpAmb}|{finNFe}|{indFinal}|{indPres}|{indIntermed}|{procEmi}|4.0.2.2|||'

                linhas.append(linha_inicial)
                linhas.append(linha_A)
                linhas.append(linha_B)
                linhas.append(linha_BA)
                linhas.append(linha_BA02)
                linhas.append(linha_C)
                linhas.append(linha_C02)
                linhas.append(linha_C05)
                linhas.append(linha_E)
                linhas.append(linha_E02)
                linhas.append(linha_E05)
                
                num_item = 1

        # 13ª linha

        linhas.append('V01||')
        linhas.append('W|')
        linhas.append('W02|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|0.00|')

        # 15ª linha
        linhas.append('X|9|')

        # 16ª linha

        tPag = 90
        vPag = 0

        linhas.append('Y|')
        linhas.append('YA|')
        linhas.append(f'YA01||{tPag}||{vPag}|')
        linhas.append('Z|||||||||||||||||||||||')
        linhas.append('ZD|||||||')
        # Gerando arquivo
        with open(os.path.join(diretorio_final, f'NF_{nNf}_Loja_{loja}.txt'), 'w') as arquivo:
            arquivo.write('\n'.join(linhas) + '\n')

        
        # Incrementa o último número utilizado para o próximo arquivo
        novo_numero_utilizado = ultimo_numero_utilizado + 1

        # Gera o novo arquivo com o número
        nome_novo_arquivo = f'C:/Bots/Bot_transmissao/Geracao CAT/Loja_{loja}/ultimo_numero_utilizado.txt'
        
        # Escreve o novo número utilizado de volta no arquivo
        escrever_ultimo_numero_utilizado(novo_numero_utilizado)
        
# Incrementa o último número utilizado para o próximo arquivo
nova_serie_utilizada = ultima_serie_utilizada - 1

# Gera o novo arquivo com o número
nome_nova_serie = 'C:/Bots/Bot_transmissao/Geracao CAT/ultima_serie_utilizada.txt'

# Escreve o novo número utilizado de volta no arquivo
escrever_ultima_serie_utilizada(nova_serie_utilizada)

C:\Users\vinic\AppData\Local\Temp\ipykernel_14264\1795614597.py:3: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  formatted_value = locale.format('%.2f', value, grouping=True)


35191027849963000110550010005826231005807695
35191027849963000110550010005827981005809503
35191027849963000110550010005834001005815514
35191027849963000110550010005835661005817174
35191027849963000110550010005836171005817680
35191027849963000110550010005836951005818461
35191027849963000110550010005839481005820969
35191027849963000110550010005841641005823091
35191027849963000110550010005845111005826595
35191027849963000110550010005845971005827456
35191027849963000110550010005846751005828230
35191027849963000110550010005846791005828271
35191027849963000110550010005847621005829101
35191027849963000110550010005847891005829377
35191027849963000110550010005848941005830419
35191027849963000110550010005855621005837132
35191027849963000110550010005857891005839423
35191027849963000110550010005857901005839432
35191027849963000110550010005857911005839448
35191027849963000110550010005857921005839453
35191027849963000110550010005857931005839469
35200927849963000110550010009230281009199079
3519102784

35191127849963000110550010006183431006164130
35191127849963000110550010006183991006164690
35191127849963000110550010006184611006165318
35191127849963000110550010006184731006165432
35191127849963000110550010006185041006165740
35191127849963000110550010006188941006169502
35191127849963000110550010006192071006172623
35191127849963000110550010006197451006178000
35191127849963000110550010006198031006178587
35191127849963000110550010006198581006179131
35191127849963000110550010006199771006180327
35191127849963000110550010006201691006182246
35191127849963000110550010006204611006185162
35191127849963000110550010006208461006189007
35191127849963000110550010006209161006189704
35191127849963000110550010006210261006190801
35191127849963000110550010006218001006198516
35191127849963000110550010006218311006198822
35191127849963000110550010006219431006199949
35191127849963000110550010006220861006201377
35191127849963000110550010006229571006210039
35191127849963000110550010006229881006210329
3519112784

35191227849963000110550010006548351006527696
35191227849963000110550010006549141006528485
35191227849963000110550010006549171006528517
35191227849963000110550010006549641006528983
35191227849963000110550010006551931006531278
35191227849963000110550010006552361006531714
35191227849963000110550010006554811006534167
35191227849963000110550010006555311006534667
35191227849963000110550010006555361006534710
35191227849963000110550010006556481006535836
35191227849963000110550010006566941006546290
35191227849963000110550010006572411006551765
35191227849963000110550010006575191006554549
35191227849963000110550010006575371006554725
35191227849963000110550010006575881006555232
35191227849963000110550010006576281006555639
35191227849963000110550010006576781006556130
35191227849963000110550010006577111006556463
35191227849963000110550010006578041006557398
35191227849963000110550010006578151006557502
35191227849963000110550010006581901006561276
35191227849963000110550010006584311006563681
3519122784

35200223276873000127550010002250551673858250
35200127849963000110550010006876121006853938
35200127849963000110550010006876181006853990
35200127849963000110550010006877661006855490
35200227849963000110550010006885391006863182
35200227849963000110550010006887391006865151
35200227849963000110550010006887661006865426
35200227849963000110550010006889491006867259
35200223276873000127550010002230191707385299
35200427849963000110550010007456311007432400
35200427849963000110550010007459611007435693
35200427849963000110550010007469411007445496
35200427849963000110550010007472851007448933
35200427849963000110550010007474091007450171
35200427849963000110550010007475061007451147
35200427849963000110550010007480151007456218
35200427849963000110550010007480531007456596
35200427849963000110550010007487361007463387
35200427849963000110550010007489791007465817
35200427849963000110550010007490011007466034
35200427849963000110550010007496431007472444
35200427849963000110550010007496981007472990
3520042784

...

35200927849963000110550010009106421009075676
35200927849963000110550010009106451009075708
35200927849963000110550010009117831009087081
35200927849963000110550010009118761009088011
35200927849963000110550010009118771009088027
35200927849963000110550010009120881009090131
35200927849963000110550010009123871009093121
35200927849963000110550010009132021009101290
35200927849963000110550010009132031009101301
35200927849963000110550010009134271009103463
35200927849963000110550010009137061009106256
35200927849963000110550010009137071009106261
35200927849963000110550010009141831009111021
35200927849963000110550010009141841009111037
35200927849963000110550010009152891009122094
35200927849963000110550010009153651009122853
35200927849963000110550010009158061009127266
35200927849963000110550010009159161009128361
35200927849963000110550010009161911009131112
35200927849963000110550010009162151009131353
35200927849963000110550010009162371009131571
35200927849963000110550010009162531009131737
3520092784

35201127849963000110550010009827551009794160
35201127849963000110550010009843741009810176
35201127849963000110550010009845411009811814
35201127849963000110550010009845861009812260
35201127849963000110550010009848651009815052
35201127849963000110550010009850571009816970
35201127849963000110550010009854451009820854
35201127849963000110550010009861121009827511
35201127849963000110550010009863281009829655
35201127849963000110550010009865251009831608
35201127849963000110550010009865311009831664
35201127849963000110550010009877381009843685
35201127849963000110550010009878901009845071
35201127849963000110550010009882211009848385
35201127849963000110550010009883911009850088
35201127849963000110550010009876511009842812
35201127849963000110550010009888571009854747
35201127849963000110550010009893471009859588
35201127849963000110550010009895131009861240
35201127849963000110550010009906551009872534
35201127849963000110550010009908111009874088
35201127849963000110550010009912061009878026
3520112784

35201027849963000110550010009583351009550781
35201027849963000110550010009584031009551461
35201027849963000110550010009584851009552285
35201027849963000110550010009587021009554451
35201027849963000110550010009588531009555967
35201027849963000110550010009589151009556585
35201027849963000110550010009589721009557154
35201027849963000110550010009590691009558126
35201027849963000110550010009592201009559637
35201027849963000110550010009596901009564327
35201027849963000110550010009598731009566150
35201027849963000110550010009598741009566165
35201027849963000110550010009599781009567204
35201027849963000110550010009602101009569523
35201027849963000110550010009605751009573173
35201027849963000110550010009607101009574524
35201027849963000110550010009612971009580353
35201027849963000110550010009613151009580531
35201027849963000110550010009614481009581879
35201027849963000110550010009621351009588741
35201027849963000110550010009622351009589743
35201027849963000110550010009624441009591837
3520102784

35191027849963000110550010005809531005790969
35191027849963000110550010005812091005793508
35191027849963000110550010005816961005798388
35191027849963000110550010005816971005798393
35191027849963000110550010005820201005801625
35191027849963000110550010005820211005801630
35191027849963000110550010005822601005804027
35191027849963000110550010005823281005804700
35191027849963000110550010005721911005703597
35191027849963000110550010005754921005736522
35191027849963000110550010005784071005765657
35191027849963000110550010005795071005776543
35191027849963000110550010005802381005783810
35191027849963000110550010005816411005797839
35191027849963000110550010005850851005832316
35191027849963000110550010005862971005844515
35191027849963000110550010005937871005919273
35191027849963000110550010005973991005955374
35191027849963000110550010005982271005963610
35191027849963000110550010005995681005976707
35191127849963000110550010006030451006011475
35191127849963000110550010006132461006113229
3519112784

35201027849963000110550010009662861009630112
35201027849963000110550010009664221009631473
35201027849963000110550010009676171009643345
35201027849963000110550010009680431009647601
35201123276873000127550010002681801791777824
35201127849963000110550010009686461009653553
35201127849963000110550010009688041009655131
35201127849963000110550010009703441009670381
35201127849963000110550010009716751009683685
35201127849963000110550010009753701009720543
35201127849963000110550010009888371009854558
35201127849963000110550010009903701009869738
35201127849963000110550010009921541009887510
35201127849963000110550010009965021009930567
35201227849963000110550010010013461009979891
35201227849963000110550010010044121010010409
35201227849963000110550010010058171010024330
35201227849963000110550010010092941010058832
35201227849963000110550010010107271010073146
35201227849963000110550010010139631010105316
35201227849963000110550010010152451010118135
35201227849963000110550010010163931010129606
3520122784

In [110]:
# Função para extrair informações de um arquivo TXT
def extrair_informacoes_arquivo(nome_arquivo):
    # Inicializa contadores
    num_linhas = 0
    num_produtos = 0
    # Lê o arquivo linha por linha
    with open(nome_arquivo, 'r') as arquivo:
        for linha in arquivo:
            num_linhas += 1
            if linha.startswith('H|'):
                # Extrai o número do produto
                numero_produto = int(linha.split('|')[1])
                # Atualiza o contador de produtos
                num_produtos = max(num_produtos, numero_produto)

    return num_linhas, num_produtos

# Lista de arquivos na pasta
pasta = f'C:/Bots/Bot_transmissao/Geracao CAT/Loja_{loja}/TXTs_complementar'
arquivos_txt = [arquivo for arquivo in os.listdir(pasta) if arquivo.endswith('.txt')]

# Lista para armazenar as informações
informacoes = []

# Loop sobre os arquivos para extrair informações
for arquivo in arquivos_txt:
    # Extrai informações do arquivo
    num_linhas, num_produtos = extrair_informacoes_arquivo(os.path.join(pasta, arquivo))
    # Adiciona as informações à lista
    informacoes.append({'Nome arquivo': arquivo, 
                        'Quantidade de linhas': num_linhas,
                        'Quantidade de produtos': num_produtos,
                       })

# Cria um DataFrame pandas com as informações
df = pd.DataFrame(informacoes)

df

,Nome arquivo,Quantidade de linhas,Quantidade de produtos
0,NF_1000_Loja_1.txt,29,1
1,NF_1001_Loja_1.txt,47,3
2,NF_1002_Loja_1.txt,560,60
3,NF_1003_Loja_1.txt,218,22
4,NF_1004_Loja_1.txt,29,1
...,...,...,...
2240,NF_997_Loja_1.txt,93,8
2241,NF_998_Loja_1.txt,49,3
2242,NF_999_Loja_1.txt,1671,173
2243,NF_99_Loja_1.txt,47,3


In [25]:
# Salva o DataFrame em uma planilha Excel
df.to_excel(f'C:/Bots/Bot_transmissao/Geracao CAT/Loja_{loja}/TXTs_complementar/Controle_Loja_{loja}.xlsx', index=False)